In [160]:
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [161]:
def step1_create_region_table(data_filename, normalized_database_filename):
    data = []
    with open(data_filename) as file:
        for line in file:
            data.append(line.split(';'))
    my_region = []
    my_holder = []
    for i in range(1,len(data)):
        cust = data[i][0].split('\t')
        if cust[4] in my_holder:
            continue
        
        my_holder.append(cust[4])
    my_region.append(tuple(my_holder))
    my_region = sorted([(my_region,) for my_region in my_holder])
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    create_table_sql = """CREATE TABLE IF NOT EXISTS [Region] (
        [RegionID] INTEGER NOT NULL PRIMARY KEY,
        [Region] TEXT NOT NULL
    );
    """
    create_table(conn, create_table_sql, True)
    sql = ''' INSERT OR REPLACE INTO Region (Region) VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, my_region)
    conn.commit()
    conn.close()
def step2_create_region_to_regionid_dictionary(normalized_database_filename):
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute("SELECT * FROM Region")
    rows = cur.fetchall()
    mydict = {}
    for i in range(len(rows)):
        mydict[rows[i][1]] = rows[i][0]
    return mydict

def step3_create_country_table(data_filename, normalized_database_filename):
    data = []
    with open(data_filename) as file:
        for line in file:
            data.append(line.split(';'))
    countries = []
    for i in range(1,len(data)):
        country = data[i][0].split('\t')[3]
        region_id = a[data[i][0].split('\t')[4]]
        if (country, region_id) in countries:
            continue
        countries.append((country, region_id))
    countries = sorted(countries)

    db_file = normalized_database_filename
    conn = create_connection(db_file)
    create_table_sql = """CREATE TABLE IF NOT EXISTS [Country] (
        [CountryID] INTEGER NOT NULL PRIMARY KEY,
        [Country] TEXT NOT NULL,
        [RegionID] INTEGER NOT NULL,
        FOREIGN KEY(RegionID) REFERENCES Region(RegionID)
    );
    """
    create_table(conn, create_table_sql)
    sql = ''' INSERT OR REPLACE INTO Country (Country, RegionID) VALUES(?, ?) '''
    cur = conn.cursor()
    cur.executemany(sql, countries)
    conn.commit()
    conn.close()
    
def step4_create_country_to_countryid_dictionary(normalized_database_filename):
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute("SELECT * FROM Country")
    rows = cur.fetchall()
    mydict = {}
    for i in rows:
        mydict[i[1]] = i[0]
    return mydict

def step5_create_customer_table(data_filename, normalized_database_filename):
    data = []
    cust = []
    a = step4_create_country_to_countryid_dictionary(normalized_database_filename)
    with open('data.csv') as file:
        for line in file:
            data.append(line.split('\n'))
    for i in range(1, len(data)):
        fname = data[i][0].split('\t')[0].split()[0] #first name
        if len(data[i][0].split('\t')[0].split()) > 2:
            lname = ' '.join(data[i][0].split('\t')[0].split()[1:]) #last name
        else:
            lname = data[i][0].split('\t')[0].split()[1]
        addy = data[i][0].split('\t')[1] #addy 
        city = data[i][0].split('\t')[2] 
        countryid = a[data[i][0].split('\t')[3]]
        cust.append((fname, lname, addy, city, countryid))
    cust = sorted(cust)   
    db_file = normalized_database_filename
    conn = create_connection(db_file)  
    create_table_sql = """CREATE TABLE IF NOT EXISTS [Customer] (
        [CustomerID] INTEGER NOT NULL PRIMARY KEY,
        [FirstName] TEXT NOT NULL,
        [LastName] TEXT NOT NULL,
        [Address] TEXT NOT NULL,
        [City] TEXT NOT NULL,
        [CountryID] INTEGER NOT NULL,
        FOREIGN KEY(CountryID) REFERENCES Country(CountryID)
    );
    """

    create_table(conn, create_table_sql)
    sql = ''' INSERT OR REPLACE INTO Customer (FirstName, LastName, Address, City, CountryID) VALUES(?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.executemany(sql, cust)
    conn.commit()
    conn.close()
def step6_create_customer_to_customerid_dictionary(normalized_database_filename):
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute("SELECT CustomerID, FirstName, LastName FROM Customer")
    rows = cur.fetchall()
    mydict = {}
    for i in range(len(rows)):
        if len(rows[i]) > 3:
            mydict[rows[i][1] + ' ' + rows[i][2] + ' ' + rows[i][3]] = rows[i][0]
        else:
            mydict[rows[i][1] + ' ' + rows[i][2]] = rows[i][0]
    return mydict
def step7_create_productcategory_table(data_filename, normalized_database_filename):
    prods = []
    data = []
    with open('data.csv') as file:
        for line in file:
            data.append(line.split('\n'))
    for i in range(1, len(data)):
        prods_cat = data[i][0].split('\t')[6].split(';')
        prods_cat_desc = data[i][0].split('\t')[7].split(';')
        for j in range(len(prods_cat)):
            if (prods_cat[j], prods_cat_desc[j]) in prods:
                continue
            prods.append((prods_cat[j], prods_cat_desc[j]))
    prods = sorted(prods)
    db_file = 'norm.db'
    conn = create_connection(db_file)  

    create_table_sql = """CREATE TABLE IF NOT EXISTS [ProductCategory] (
        [ProductCategoryID] INTEGER NOT NULL PRIMARY KEY,
        [ProductCategory] TEXT NOT NULL,
        [ProductCategoryDescription] TEXT NOT NULL
    );"""
    create_table(conn, create_table_sql)
    sql = ''' INSERT OR REPLACE INTO ProductCategory (ProductCategory, ProductCategoryDescription) VALUES(?, ?) '''
    cur = conn.cursor()
    cur.executemany(sql, prods)
    conn.commit()
    conn.close()

def step8_create_productcategory_to_productcategoryid_dictionary(normalized_database_filename):
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute("SELECT ProductCategoryID, ProductCategory FROM ProductCategory")
    rows = cur.fetchall()
    mydict = {}
    for i in range(len(rows)):
        mydict[rows[i][1]] = rows[i][0]
    return mydict

def step9_create_product_table(data_filename, normalized_database_filename):
    data = []
    mydict = step8_create_productcategory_to_productcategoryid_dictionary(normalized_database_filename)
    with open(data_filename) as file:
        for line in file:
            data.append(line.split('\n'))
    products = []
    for i in range(1, len(data)):
        prod_name_list = data[i][0].split('\t')[5].split(';')
        prod_cat_list = data[i][0].split('\t')[6].split(';')
        prod_price_list = data[i][0].split('\t')[8].split(';')
        for j in range(len(prod_name_list)):
            if (prod_name_list[j], float(prod_price_list[j]), mydict[prod_cat_list[j]]) in products:
                continue
            products.append((prod_name_list[j], float(prod_price_list[j]), mydict[prod_cat_list[j]]))
    products = sorted(products)
    db_file = normalized_database_filename
    conn = create_connection(db_file)

    
    
      # Create the ProductCategory table if it doesn't exist
    create_table_sql = """CREATE TABLE IF NOT EXISTS [Product] (
          [ProductID] INTEGER NOT NULL PRIMARY KEY,
          [ProductName] TEXT NOT NULL,
          [ProductUnitPrice] REAL NOT NULL,
          [ProductCategoryID] INTEGER NOT NULL,
          FOREIGN KEY(ProductCategoryID) REFERENCES ProductCategory(ProductCategoryID)

      );"""
    create_table(conn, create_table_sql)
    sql = ''' INSERT OR REPLACE INTO Product (ProductName, ProductUnitPrice, ProductCategoryID) VALUES (?, ?, ?) '''
    cur = conn.cursor()
    cur.executemany(sql, products)
    conn.commit()
    conn.close()

def step10_create_product_to_productid_dictionary(normalized_database_filename):
    db_file = normalized_database_filename
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute("SELECT ProductID, ProductName FROM Product")
    rows = cur.fetchall()
    mydict = {}
    for i in rows:
        if i in mydict:
            continue
        else:
            mydict[i[1]] = i[0] 
    return mydict 

def step11_create_orderdetail_table(data_filename, normalized_database_filename):
    data = []
    with open('data.csv') as file:
        for line in file:
            data.append(line.split('\n'))
    sql_inserter = tuple()
    cust_list = step6_create_customer_to_customerid_dictionary('norm.db')
    prod_list = step10_create_product_to_productid_dictionary('norm.db')
    for i in range(1,len(data)):
        print(i)
        length = len(data[i][0].split('\t')[5].split(';'))
        name = data[i][0].split('\t')[0]
        cust_id = tuple([cust_list[name]]*length)
        prod_id = list(map(lambda x: prod_list[x], data[i][0].split('\t')[5].split(';')))
        date_strings = data[i][0].split('\t')[10].split(';')
        formatted_dates = [(datetime.strptime(date_str, '%Y%m%d')).strftime('%Y-%m-%d') for date_str in date_strings]
        formatted_dates = tuple(formatted_dates)
        quat = tuple(int(element) for element in data[i][0].split('\t')[9].split(';'))
        sql_inserter += tuple(zip(cust_id, prod_id, formatted_dates, quat))
    sql_inserter = list(sql_inserter)



    db_file = 'norm.db'
    conn = create_connection(db_file)



    # Create the ProductCategory table if it doesn't exist
    create_table_sql = """CREATE TABLE IF NOT EXISTS [OrderDetail] (
              [OrderID] INTEGER NOT NULL PRIMARY KEY,
              [CustomerID] INTEGER NOT NULL,
              [ProductID] INTEGER NOT NULL,
              [OrderDate] INTEGER NOT NULL,
              [QuantityOrdered] INTEGER NOT NULL,            
              FOREIGN KEY(CustomerID) REFERENCES Customer(CustomerID),
              FOREIGN KEY(ProductID) REFERENCES Product(ProductID)

            );"""
    create_table(conn, create_table_sql)
    sql = ''' INSERT OR REPLACE INTO OrderDetail (CustomerID, ProductID, OrderDate, QuantityOrdered) VALUES (?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.executemany(sql, sql_inserter)
    conn.commit()
    conn.close()
    

In [ ]:
def ex1(conn, CustomerName):
  cursor = conn.cursor()
  customer_id = step6_create_customer_to_customerid_dictionary('normalized.db')[CustomerName]
  sql_query = """
        SELECT 
            Customer.FirstName || ' ' || Customer.LastName AS Name,
            Product.ProductName,
            OrderDetail.OrderDate,
            Product.ProductUnitPrice,
            OrderDetail.QuantityOrdered,
            ROUND((Product.ProductUnitPrice * OrderDetail.QuantityOrdered), 2) AS Total
        FROM 
            OrderDetail
        JOIN 
            Customer ON OrderDetail.CustomerID = Customer.CustomerID
        JOIN 
            Product ON OrderDetail.ProductID = Product.ProductID
        WHERE 
            Customer.CustomerID = {}
            """.format(customer_id)
  return sql_query
    
    # Simply, you are fetching all the rows for a given CustomerName. 
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer and Product table.
    # Pull out the following columns. 
    # Name -- concatenation of FirstName and LastName
    # ProductName
    # OrderDate
    # ProductUnitPrice
    # QuantityOrdered
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- round to two decimal places
    # HINT: USE customer_to_customerid_dict to map customer name to customer id and then use where clause with CustomerID
    
    
# WRITE YOUR CODE HERE

def ex2(conn, CustomerName):
  cursor = conn.cursor()
  customer_id = step6_create_customer_to_customerid_dictionary('normalized.db')[CustomerName]
  sql_statement = """
        SELECT 
            Customer.FirstName || ' ' || Customer.LastName AS Name,
            ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered), 2) AS Total
        FROM 
            OrderDetail
        JOIN 
            Customer ON OrderDetail.CustomerID = Customer.CustomerID
        JOIN 
            Product ON OrderDetail.ProductID = Product.ProductID
        WHERE 
            Customer.CustomerID = {}
        GROUP BY 
            Customer.FirstName, Customer.LastName
    """.format(customer_id)
  return sql_statement
    
    # Simply, you are summing the total for a given CustomerName. 
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer and Product table.
    # Pull out the following columns. 
    # Name -- concatenation of FirstName and LastName
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- sum first and then round to two decimal places
    # HINT: USE customer_to_customerid_dict to map customer name to customer id and then use where clause with CustomerID
    

# WRITE YOUR CODE HERE
    

def ex3(conn):  
    # Simply, find the total for all the customers
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer and Product table.
    # Pull out the following columns. 
    # Name -- concatenation of FirstName and LastName
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- sum first and then round to two decimal places
    # ORDER BY Total Descending 
    
  sql_statement = """
            SELECT 
            Customer.FirstName || ' ' || Customer.LastName AS Name,
            ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered), 2) AS Total
        FROM 
            OrderDetail
        JOIN 
            Customer ON OrderDetail.CustomerID = Customer.CustomerID
        JOIN 
            Product ON OrderDetail.ProductID = Product.ProductID
        GROUP BY 
            Customer.FirstName, Customer.LastName
        ORDER BY 
            Total DESC
    """
# WRITE YOUR CODE HERE
  return sql_statement

def ex4(conn):

    # Simply, find the total for all the region
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer, Product, Country, and 
    # Region tables.
    # Pull out the following columns. 
    # Region
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- sum first and then round to two decimal places
    # ORDER BY Total Descending 
    
  sql_statement = """
          SELECT 
            Region.Region AS Region,
            ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered), 2) AS Total
        FROM 
            OrderDetail
        JOIN 
            Customer ON OrderDetail.CustomerID = Customer.CustomerID
        JOIN 
            Product ON OrderDetail.ProductID = Product.ProductID
        JOIN 
            Country ON Customer.CountryID = Country.CountryID
        JOIN 
            Region ON Country.RegionID = Region.RegionID
        GROUP BY 
            Region.Region
        ORDER BY 
            Total DESC
    """
# WRITE YOUR CODE HERE
  return sql_statement

def ex5(conn):

    # Simply, find the total for all the countries
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer, Product, and Country table.
    # Pull out the following columns. 
    # Country
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- sum first and then round
    # ORDER BY Total Descending 
  sql_statement = """
          SELECT 
            Country.Country,
            ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered)) AS Total
        FROM 
            OrderDetail
        JOIN 
            Customer ON OrderDetail.CustomerID = Customer.CustomerID
        JOIN 
            Product ON OrderDetail.ProductID = Product.ProductID
        JOIN 
            Country ON Customer.CountryID = Country.CountryID
        GROUP BY 
            Country.Country
        ORDER BY 
            Total DESC
    
    """
# WRITE YOUR CODE HERE
  return sql_statement


def ex6(conn):

    # Rank the countries within a region based on order total
    # Output Columns: Region, Country, CountryTotal, TotalRank
    # Hint: Round the the total
    # Hint: Sort ASC by Region

  sql_statement = """
            WITH CountryTotalRank AS (
            SELECT 
                Region.Region,
                Country.Country AS Country,
                ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered)) AS CountryTotal,
                RANK() OVER (PARTITION BY Region.Region ORDER BY SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered) DESC) AS TotalRank
            FROM 
                OrderDetail
            JOIN 
                Customer ON OrderDetail.CustomerID = Customer.CustomerID
            JOIN 
                Product ON OrderDetail.ProductID = Product.ProductID
            JOIN 
                Country ON Customer.CountryID = Country.CountryID
            JOIN 
                Region ON Country.RegionID = Region.RegionID
            GROUP BY 
                Region.Region, Country.Country
        )
        SELECT 
            Region,
            Country,
            CountryTotal,
            TotalRank
        FROM 
            CountryTotalRank
        ORDER BY 
            Region ASC, TotalRank ASC
    """

# WRITE YOUR CODE HERE
  df = pd.read_sql_query(sql_statement, conn)
  return sql_statement



def ex7(conn):

    # Rank the countries within a region based on order total, BUT only select the TOP country, meaning rank = 1!
    # Output Columns: Region, Country, Total, TotalRank
    # Hint: Round the the total
    # Hint: Sort ASC by Region
    # HINT: Use "WITH"
  sql_statement = """
            WITH CountryTotalRank AS (
            SELECT 
                Region.Region,
                Country.Country,
                ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered)) AS CountryTotal,
                RANK() OVER (PARTITION BY Region.Region ORDER BY SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered) DESC) AS CountryRegionalRank
            FROM 
                OrderDetail
            JOIN 
                Customer ON OrderDetail.CustomerID = Customer.CustomerID
            JOIN 
                Product ON OrderDetail.ProductID = Product.ProductID
            JOIN 
                Country ON Customer.CountryID = Country.CountryID
            JOIN 
                Region ON Country.RegionID = Region.RegionID
            GROUP BY 
                Region.Region, Country.Country
        )
        SELECT 
            Region,
            Country,
            CountryTotal,
            CountryRegionalRank
        FROM 
            CountryTotalRank
        WHERE 
            CountryRegionalRank = 1
        ORDER BY 
            Region ASC
    """
# WRITE YOUR CODE HERE
  return sql_statement

def ex8(conn):

    # Sum customer sales by Quarter and year
    # Output Columns: Quarter,Year,CustomerID,Total
    # HINT: Use "WITH"
    # Hint: Round the the total
    # HINT: YOU MUST CAST YEAR TO TYPE INTEGER!!!!
  sql_statement = """
            WITH CustomerSalesByQuarter AS (
            SELECT 
                STRFTIME('%Y', OrderDetail.OrderDate) AS Year,
                CASE 
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '01' AND '03' THEN 'Q1'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '04' AND '06' THEN 'Q2'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '07' AND '09' THEN 'Q3'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '10' AND '12' THEN 'Q4'
                END AS Quarter,
                OrderDetail.CustomerID,
                ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered)) AS Total
            FROM 
                OrderDetail
            JOIN 
                Product ON OrderDetail.ProductID = Product.ProductID
            GROUP BY 
                Year, Quarter, OrderDetail.CustomerID
        )
        SELECT 
            Quarter,
            CAST(Year AS INTEGER) AS Year,
            CustomerID,
            Total
        FROM 
            CustomerSalesByQuarter
    """
# WRITE YOUR CODE HERE
  return sql_statement

def ex9(conn):
    # Rank the customer sales by Quarter and year, but only select the top 5 customers!
    # Output Columns: Quarter, Year, CustomerID, Total
    # HINT: Use "WITH"
    # Hint: Round the the total
    # HINT: YOU MUST CAST YEAR TO TYPE INTEGER!!!!
    # HINT: You can have multiple CTE tables;
    # WITH table1 AS (), table2 AS ()
  sql_statement = """
            WITH CustomerSalesByQuarter AS (
            SELECT 
                STRFTIME('%Y', OrderDetail.OrderDate) AS Year,
                CASE 
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '01' AND '03' THEN 'Q1'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '04' AND '06' THEN 'Q2'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '07' AND '09' THEN 'Q3'
                    WHEN STRFTIME('%m', OrderDetail.OrderDate) BETWEEN '10' AND '12' THEN 'Q4'
                END AS Quarter,
                OrderDetail.CustomerID,
                ROUND(SUM(Product.ProductUnitPrice * OrderDetail.QuantityOrdered)) AS Total
            FROM 
                OrderDetail
            JOIN 
                Product ON OrderDetail.ProductID = Product.ProductID
            GROUP BY 
                Year, Quarter, OrderDetail.CustomerID
        ),
        RankedCustomerSales AS (
            SELECT 
                Quarter,
                CAST(Year AS INTEGER) AS Year,
                CustomerID,
                Total,
                RANK() OVER (PARTITION BY Year, Quarter ORDER BY Total DESC) AS SalesRank
            FROM 
                CustomerSalesByQuarter
        )
        SELECT 
            Quarter,
            Year,
            CustomerID,
            Total,
            SalesRank AS CustomerRank
        FROM 
            RankedCustomerSales
        WHERE 
            SalesRank <= 5
    """
  return sql_statement

def ex10(conn):

    # Rank the monthy sales
    # Output Columns: Quarter, Year, CustomerID, Total
    # HINT: Use "WITH"
    # Hint: Round the the total

  sql_statement = """
            WITH MonthlySales AS (
            SELECT 
                STRFTIME('%m', OrderDetail.OrderDate) AS MonthNumber,
                ROUND(SUM(ROUND(Product.ProductUnitPrice * OrderDetail.QuantityOrdered))) AS Total
            FROM 
                OrderDetail
            JOIN 
                Product ON OrderDetail.ProductID = Product.ProductID
            GROUP BY 
                MonthNumber
        ),
        RankedMonthlySales AS (
            SELECT 
                MonthNumber,
                Total,
                RANK() OVER (ORDER BY Total DESC) AS TotalRank
            FROM 
                MonthlySales
        )
        SELECT 
            CASE
                WHEN MonthNumber = '01' THEN 'January'
                WHEN MonthNumber = '02' THEN 'February'
                WHEN MonthNumber = '03' THEN 'March'
                WHEN MonthNumber = '04' THEN 'April'
                WHEN MonthNumber = '05' THEN 'May'
                WHEN MonthNumber = '06' THEN 'June'
                WHEN MonthNumber = '07' THEN 'July'
                WHEN MonthNumber = '08' THEN 'August'
                WHEN MonthNumber = '09' THEN 'September'
                WHEN MonthNumber = '10' THEN 'October'
                WHEN MonthNumber = '11' THEN 'November'
                WHEN MonthNumber = '12' THEN 'December'
            END AS Month,
            ROUND(Total, 2) AS Total,
            TotalRank
        FROM 
            RankedMonthlySales
    """

# WRITE YOUR CODE HERE
  return sql_statement

def ex11(conn):

    
    # Find the MaxDaysWithoutOrder for each customer 
    # Output Columns: 
    # CustomerID,
    # FirstName,
    # LastName,
    # Country,
    # OrderDate, 
    # PreviousOrderDate,
    # MaxDaysWithoutOrder
    # order by MaxDaysWithoutOrder desc
    # HINT: Use "WITH"; I created two CTE tables
    # HINT: Use Lag
    sql_statement = """
            WITH CustomerOrderDates AS (
            SELECT
                Customer.CustomerID,
                Customer.FirstName,
                Customer.LastName,
                Country.Country AS Country,
                OrderDetail.OrderDate,
                LAG(OrderDetail.OrderDate) OVER (PARTITION BY Customer.CustomerID ORDER BY OrderDetail.OrderDate) AS PreviousOrderDate
            FROM
                OrderDetail
            JOIN
                Customer ON OrderDetail.CustomerID = Customer.CustomerID
            JOIN
                Country ON Customer.CountryID = Country.CountryID
        ),
        DaysWithoutOrder AS (
            SELECT
                CustomerID,
                FirstName,
                LastName,
                Country,
                OrderDate,
                PreviousOrderDate,
                JULIANDAY(OrderDate) - JULIANDAY(PreviousOrderDate) AS DaysWithoutOrder
            FROM
                CustomerOrderDates
            )
        SELECT
            CustomerID,
            FirstName,
            LastName,
            Country,
            OrderDate,
            PreviousOrderDate,
            MAX(DaysWithoutOrder) AS MaxDaysWithoutOrder
        FROM
            DaysWithoutOrder
        GROUP BY
        CustomerID, FirstName, LastName, Country
        ORDER BY
            MaxDaysWithoutOrder DESC, FirstName ASC

    """
# WRITE YOUR CODE HERE
    return sql_statement